In [ ]:
pip install google-generativeai requests beautifulsoup4 pandas

In [ ]:
pip install google-genai pydantic

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class TickertapeLink(BaseModel):
    """
    Defines the structure we expect from the AI model.
    """
    url: str = Field(description="The full URL to the stock's page on Tickertape.in")

In [ ]:
# Correct way to read the Gemini API key from a file
with open(r"C:\Users\Rohit Jonnadula\secret_key.txt.txt", "r") as file:
    GEMINI_API_KEY = file.read().strip()

In [ ]:
# Correct way to read the Gemini API key from a file
with open(r"C:\Users\Rohit Jonnadula\secret_key.txt.txt", "r") as file:
    GEMINI_API_KEY = file.read().strip()

In [ ]:
import google.generativeai as genai
import os

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY  # or set this in your environment
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

print("Gemini client initialized successfully.")

In [85]:
from google.genai import types
import re

def generate_link_with_gemini(ticker_symbol: str):
    # 3a. The prompt: The specific instruction for the AI.
    prompt = (
        f"get the url for nse ticker symbol '{ticker_symbol}' from tickertape."
        # f"The URL format is 'www.tickertape.in[TICKER_SYMBOL]'. "
        f"Only return the required JSON structure."
    )

    # 3b. The configuration: The rules for the output format.
    config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=TickertapeLink, # Points to our Pydantic class
    )
    
    # ... move to Step 4 ...
    print(f"Calling Gemini API for ticker: {ticker_symbol}...")
    model = genai.GenerativeModel("gemini-2.5-flash")
    response = model.generate_content(prompt)

    # 5a. The response is a JSON string (e.g., '{"url": "..."}')
    print(f"Raw AI response text: {response.text}")


    raw_response = response.text
    # Remove ```json and ```
    if raw_response.startswith('```json'):
        clean_json = raw_response[7:].strip()  # Remove ```
    if raw_response.endswith('```'):
        clean_json = clean_json[:-3].strip()   # Remove ```
    data_object = TickertapeLink.model_validate_json(clean_json)


    # 5c. Access the specific attribute (the link) and assign it to a Python variable
    final_link_variable = data_object.url

    return final_link_variable

In [87]:
ticker_to_search = "pricolltd" # Example for an Indian stock
my_final_url = generate_link_with_gemini(ticker_to_search)

print("-" * 40)
print(f"The final link assigned to the variable 'my_final_url' is:")
print(f"\n>>>> {my_final_url} <<<<\n")

Calling Gemini API for ticker: pricolltd...
Raw AI response text: ```json
{
  "url": "https://www.tickertape.in/app/stocks/pricolltd"
}
```
----------------------------------------
The final link assigned to the variable 'my_final_url' is:

>>>> https://www.tickertape.in/app/stocks/pricolltd <<<<

Raw AI response text: ```json
{
  "url": "https://www.tickertape.in/app/stocks/pricolltd"
}
```
----------------------------------------
The final link assigned to the variable 'my_final_url' is:

>>>> https://www.tickertape.in/app/stocks/pricolltd <<<<



In [ ]:
import pandas as pd
import requests
import google.generativeai as genai
from bs4 import BeautifulSoup
ticker_name = 'jkil'
url = f'https://www.tickertape.in/stocks/{ticker_name}'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify()[:1000])  # Print the first 1000 characters of the prettified HTML